In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/domain-normal/sleep_stress_dataset.json


In [ ]:
import json
import time
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer, util
import psutil
import os
import gc
from huggingface_hub import login

login("your_hf_token")

2025-10-03 15:47:48.149438: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759506468.362451      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759506468.425280      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
DATASET_PATH = "/kaggle/input/domain-normal/sleep_stress_dataset.json"

MODELS = [
    "microsoft/phi-3-mini-4k-instruct",     # Phi-3-mini (5.3B)
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",   # TinyLlama
    "google/gemma-2b",                      # Gemma (2B)
    "facebook/opt-1.3b",                    # OPT-1.3B
    "EleutherAI/pythia-2.8b",               # Pythia 2.8B    
    "tiiuae/Falcon3-1B-Instruct"            # Falcon-1B
]

EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embedder = SentenceTransformer(EMBED_MODEL)

MAX_SAMPLES = 200

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
dataset = []
with open(DATASET_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)   

for idx, sample in enumerate(data):
    query = sample["instruction"]
    if sample.get("input"):
        query += "\n" + sample["input"]
    dataset.append({
        "id": idx,
        "query": query,
        "reference": sample["output"]
    })

if MAX_SAMPLES:
    dataset = dataset[:MAX_SAMPLES]


In [5]:
def cleanup():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

In [6]:
def benchmark_model(model_name, dataset, batch_size=8):
    print(f"\n=== Benchmarking {model_name} ===")
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name, torch_dtype=torch.float16, device_map="auto"
    )
    gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    results = []

    # Process dataset in batches
    for i in range(0, len(dataset), batch_size):
        batch = dataset[i:i+batch_size]
        queries = [sample["query"] for sample in batch]
        refs = [sample["reference"] for sample in batch]

        start_time = time.time()
        process = psutil.Process(os.getpid())
        mem_before = process.memory_info().rss / (1024 * 1024)

        # Generate batch responses
        batch_responses = gen_pipeline(
            queries, max_new_tokens=128, do_sample=False, temperature=0.0
        )

        latency = time.time() - start_time
        mem_after = process.memory_info().rss / (1024 * 1024)
        mem_used = mem_after - mem_before

        for j, sample in enumerate(batch):
            # Hugging Face pipeline returns list[list[dict]]
            response_text = batch_responses[j][0]["generated_text"]

            emb_ref = embedder.encode(refs[j], convert_to_tensor=True)
            emb_resp = embedder.encode(response_text, convert_to_tensor=True)
            similarity = util.cos_sim(emb_ref, emb_resp).item()

            results.append({
                "id": sample["id"],
                "query": sample["query"],
                "reference": refs[j],
                "response": response_text,
                "similarity": round(similarity, 3),
                "latency_sec": round(latency, 3),
                "mem_used_MB": round(mem_used, 2)
            })

    # Aggregate stats
    avg_sim = sum(r["similarity"] for r in results) / len(results)
    avg_lat = sum(r["latency_sec"] for r in results) / len(results)
    avg_mem = sum(r["mem_used_MB"] for r in results) / len(results)

    print(f"Samples: {len(results)}")
    print(f"Avg Semantic Similarity: {avg_sim:.3f}")
    print(f"Avg Latency: {avg_lat:.3f} sec")
    print(f"Avg Memory Used: {avg_mem:.2f} MB")

    out_file = f"benchmark_{model_name.replace('/', '_')}.jsonl"
    with open(out_file, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    del model, tokenizer, gen_pipeline
    cleanup()

    return {
        "model": model_name,
        "avg_similarity": round(avg_sim, 3),
        "avg_latency": round(avg_lat, 3),
        "avg_memory": round(avg_mem, 2)
    }

In [7]:
leaderboard = []
for model_name in MODELS:
    summary = benchmark_model(model_name, dataset)
    leaderboard.append(summary)


=== Benchmarking microsoft/phi-3-mini-4k-instruct ===


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Samples: 200
Avg Semantic Similarity: 0.678
Avg Latency: 28.366 sec
Avg Memory Used: -1.88 MB


In [8]:
print("\n=== Leaderboard ===")
print(f"{'Model':40s} | {'Similarity':>10s} | {'Latency (s)':>10s} | {'Memory (MB)':>12s}")
print("-"*80)
for row in leaderboard:
    print(f"{row['model']:40s} | {row['avg_similarity']:10.3f} | {row['avg_latency']:10.3f} | {row['avg_memory']:12.2f}")



=== Leaderboard ===
Model                                    | Similarity | Latency (s) |  Memory (MB)
--------------------------------------------------------------------------------
microsoft/phi-3-mini-4k-instruct         |      0.713 |     43.427 |        11.00
TinyLlama/TinyLlama-1.1B-Chat-v1.0       |      0.720 |      2.168 |       -29.25
google/gemma-2b                          |      0.564 |     31.654 |         0.03
facebook/opt-1.3b                        |      0.664 |     20.943 |        -9.79
EleutherAI/pythia-2.8b                   |      0.663 |     34.622 |        -0.05
tiiuae/Falcon3-1B-Instruct               |      0.678 |     28.366 |        -1.88
